In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
import keras
import scipy as sp
import scipy.signal
import tensorflow as tf
import tensorflow.keras.losses as losses
import tensorflow.keras.optimizers as optimizers
import scipy.io as sio
import numpy as np
import sklearn.metrics as metrics 
import matplotlib.pyplot as plt
import statistics

from tensorflow_addons.metrics import RSquare
from tensorflow.keras.models import Sequential
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from wandb.keras import WandbCallback
import wandb
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

2023-04-08 14:26:29.427352: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-08 14:26:32.817632: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib64:/home/malfonzo/miniconda3/lib/
2023-04-08 14:26:32.817711: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib64:/home/malfonzo/miniconda3/lib/
2023-04-08 14:

In [2]:
F1_test=8
D_test=2
F2_test=16
Drop_test=0.25 #originally 0.5
KernLength_test=64 #Originally 64 
batch_test=16

N_chan=31
N_samples_long=251
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-04-08 14:26:39.275296: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-08 14:26:39.305868: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-08 14:26:39.306194: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [10]:
# PER SUB 
# SUBJECTS=[1,2,3,4,5,6,7,8,9,11,12,13,14,16,17]
# for temp_sub in SUBJECTS:
CLASSES= [2,3]
#questa va applicata a for loop di subject che deve essere il più esterno 
sub="{:02d}".format(temp_sub)
for n_classes in CLASSES:
    #Define loss function
    loss_fn= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    LABELS= list(range(0,n_classes))
    numero_classi=n_classes

#     dir0= os.path.join("FINAL_motor_imagery_classification/2_sec_windows/raw/"+str(n_classes)+"class/output/", "sub_"+str(sub))
    dir0= os.path.join("FINAL_motor_imagery_classification/2_sec_windows/raw/"+str(n_classes)+"class/output/", "all")

#     os.mkdir(dir0)
    dir_input=("FINAL_motor_imagery_classification/2_sec_windows/raw/"+str(n_classes)+"class/input")

    evaluation=[]
    iteration=[]
    confusion_matrix_x_test=[]
    confusion_matrix_y_test= []
    validation_acc=[]
    PERFORMANCE=[]

#     print("SUBJECT: "+ str(sub))
    print("N_CLASSES: "+ str(n_classes))

#     X=sio.loadmat(dir_input+"/sub_"+str(sub)+ "_X.mat")["array_X"]
#     y= sio.loadmat(dir_input+"/sub_"+str(sub)+"_y.mat")["array_y"]

    X=sio.loadmat(dir_input+"/all_X.mat")["array_X"]
    y= sio.loadmat(dir_input+"/all_y.mat")["array_y"]
    X= X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
    y=y.flatten()
    print("New shape for X: " + str(X.shape))
    print("New shape for y: "+str(y.shape))

    dir1=os.path.join(dir0, "comparison")
    dir2=os.path.join(dir0, "temporal_convolution")
    dir3=os.path.join(dir0, "spatial_convolution")
#     os.mkdir(dir1)
#     os.mkdir(dir2)
#     os.mkdir(dir3)

    ################################################################################################################################################################################################################################################################################

    n_folds = 5
    seed = 21
    shuffle_test = True
    EPOCHS=250

    kfold = KFold(n_splits = n_folds, shuffle = shuffle_test, random_state = seed)
    count=0
    sommatoria=0

    for train_index, test_index in kfold.split(X):
        count=count+1

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        print("Train index for this split: "+ str(train_index)) 
        print("Number of samples for train set: "+str(train_index.shape[0]))
        print("Test index for this split: "+ str(test_index))
        print("Number of samples for test set: "+str(test_index.shape[0]))

        # Define the model architecture - 

        model=Sequential()

        ##################################################################

        model.add(Conv2D(8, (1, 64), padding = 'same',
                                       input_shape = (N_chan, N_samples_long, 1),
                                       use_bias = False, name="temporal_conv"))
        model.add(BatchNormalization(name="batchnorm_1"))
        model.add(DepthwiseConv2D((31, 1), use_bias = False, 
                                       depth_multiplier = 2,
                                       depthwise_constraint = max_norm(1.), name="spatial_conv"))
        model.add(BatchNormalization(name="batchnorm_2"))
        model.add(Activation('elu', name="activation_1"))
        model.add(AveragePooling2D((1, 4), name="pooling_layer_1"))
        model.add(Dropout(0.5, name="dropout_1"))

        model.add(SeparableConv2D(16, (1, 16),
                                       use_bias = False, padding = 'same', name="separable_conv"))
        model.add(BatchNormalization(name="batchnorm_3"))
        model.add(Activation('elu', name="activation_2"))
        model.add(AveragePooling2D((1, 8), name="pooling_layer_2"))
        model.add(Dropout(0.5, name="drpout_2")) #QUI DROPOUT E' LASCIATO A 0.5 come in eegnet paper

        model.add(Flatten(name = 'flatten'))

        model.add(Dense(numero_classi, name = 'dense', 
                                 kernel_constraint = max_norm(0.25)))
        model.add(Activation('softmax', name = 'softmax'))


        # Define the optimizer
        optimizer= optimizers.Adam(
        learning_rate= 1e-3,
        weight_decay= 0
        )
        model.compile(optimizer=optimizer,
                       loss=loss_fn,
                       metrics=['accuracy'])

        evaluation.append(model.fit(X_train, y_train, batch_size=16,
                  epochs=EPOCHS, 
                  validation_data=(X_test, y_test), 
                  verbose=2, workers=1)
                     )

        # Iteration = fold, i am just saving the model for that fold
        iteration.append(model)

        confusion_matrix_x_test.append(X_test)
        confusion_matrix_y_test.append(y_test)

        #Plotting confusion matrix
        pred=model.predict(X_test)
        y_test_pred= np.argmax(pred, axis=1)

        confusion_matrix= metrics.confusion_matrix(y_test, y_test_pred, normalize='true')
        plt.figure()
        metrics.ConfusionMatrixDisplay(confusion_matrix).plot()
        plt.savefig(dir1+"/confusion_matrix_kfold_"+str(count))
        plt.close()

        validation_acc.append(np.sum(y_test==y_test_pred)/y_test.shape[0])

        PERFORMANCE.append(classification_report(y_test, y_test_pred, labels=LABELS, output_dict=True))

        #Salvo risultati di singolo fold
        sio.savemat(dir1+"/y_pred_test_kfold"+str(count), {"array": y_test_pred})
        sio.savemat(dir1+"/y_test_kfold"+str(count), {"array": y_test})


        # PLOTTO FILTRI TEMPORALI E SPAZIALI E LI SALVO
        var= (model.get_layer("temporal_conv").weights)
        for lallo in range(8):
            plt.figure()
            plt.title("temp_conv_"+str(lallo))
            plt.plot(var[0][0,:,0][:,lallo]) #this way i access the temporal filters, cambiando ultimo zero
            plt.savefig(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo))
            temp= (var[0][0,:,0][:,lallo]).numpy()
            sio.savemat(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": temp})
            plt.close()


        var_2= (model.get_layer("spatial_conv").weights)
        reshaped_var_2=tf.reshape(var_2[0][:,0,:,:],(31,16))
        for lallo in range(16):
            nump= reshaped_var_2[:,lallo].numpy()
            sio.savemat(dir3+"/spat_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": nump})

    ###################################################################################################################

    #plot accuracy and loss function across epochs
    epoch_vec=np.linspace(1,EPOCHS,EPOCHS)
    min_temp_loss=10
    min_temp_acc=10
    max_temp_loss=0
    max_temp_acc=0

    for idx in range(n_folds):
        if (np.min(evaluation[idx].history['loss'])<min_temp_loss):
            min_temp_loss=np.min(evaluation[idx].history['loss'])
        if (np.min(evaluation[idx].history['val_loss'])<min_temp_loss):
            min_temp_loss=np.min(evaluation[idx].history['val_loss'])
        if (np.max(evaluation[idx].history['loss'])>max_temp_loss):
            max_temp_loss=np.max(evaluation[idx].history['loss'])
        if (np.max(evaluation[idx].history['val_loss'])>max_temp_loss):
            max_temp_loss=np.max(evaluation[idx].history['val_loss'])

    for idx in range(n_folds):
        if (np.min(evaluation[idx].history['accuracy'])<min_temp_acc):
            min_temp_acc=np.min(evaluation[idx].history['accuracy'])
        if (np.min(evaluation[idx].history['val_accuracy'])<min_temp_acc):
            min_temp_acc=np.min(evaluation[idx].history['val_accuracy'])
        if (np.max(evaluation[idx].history['accuracy'])>max_temp_acc):
            max_temp_acc=np.max(evaluation[idx].history['accuracy'])
        if (np.max(evaluation[idx].history['val_accuracy'])>max_temp_acc):
            max_temp_acc=np.max(evaluation[idx].history['val_accuracy'])

    for idx in range(n_folds):
        loss_vec_train= evaluation[idx].history['loss']
        loss_vec_test= evaluation[idx].history['val_loss']

        plt.figure()
        plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
        plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Loss across epochs for fold: '+str(idx))
        plt.ylim([min_temp_loss, max_temp_loss])
        plt.legend()

        plt.savefig(dir1+"/loss_kfold_"+str(idx))
        plt.close()

    #plot accuracy and loss function across epochs
    epoch_vec=np.linspace(1,EPOCHS,EPOCHS)

    for idx in range(n_folds):
        loss_vec_train= evaluation[idx].history['accuracy']
        loss_vec_test= evaluation[idx].history['val_accuracy']

        plt.figure()
        plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
        plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.title('Accuracy across epochs for fold: '+str(idx))
        plt.ylim([min_temp_acc, max_temp_acc])
        plt.legend()

        plt.savefig(dir1+"/accuracy_kfold_"+str(idx))
        plt.close()

    #SALVO VARIABILI STATISTICHE E MODELLO IN MODO DA PLOTTARLO

#         acc_temp=[]
    accuratezza=np.zeros(n_classes*2)
#         for idx in range(n_folds):
#             acc_temp.append(PERFORMANCE[idx]["accuracy"])
    accuratezza[0]=(np.mean(validation_acc))
    accuratezza[1]=(statistics.pstdev(validation_acc))


    precisione=[]
    recall=[]
    f1_score=[]
    support=[]
    for classe in range(numero_classi):
        precision_temp=[]
        recall_temp=[]
        f1_score_temp=[]
        support_temp=[]
        for idx in range(n_folds):
            precision_temp.append(PERFORMANCE[idx][str(classe)]["precision"])
            recall_temp.append(PERFORMANCE[idx][str(classe)]["recall"])
            f1_score_temp.append(PERFORMANCE[idx][str(classe)]["f1-score"])
            support_temp.append(PERFORMANCE[idx][str(classe)]["support"])

        precisione.append(np.mean(precision_temp))
        precisione.append(statistics.pstdev(precision_temp))
        recall.append(np.mean(recall_temp))
        recall.append(statistics.pstdev(recall_temp))
        f1_score.append(np.mean(f1_score_temp))
        f1_score.append(statistics.pstdev(f1_score_temp))    
        support.append(np.mean(support_temp))
        support.append(statistics.pstdev(support_temp)) 

    sommario=[]
    sommario=np.vstack((accuratezza,precisione,recall,f1_score,support))
    sio.savemat(dir0+"/sommario.mat", {"array": sommario})

    gianfranco=model.predict(X)
    gianfranco2=np.argmax(gianfranco, axis=1)
    sio.savemat(dir0+"/predizione_totale.mat", {"array": gianfranco2})



N_CLASSES: 2
New shape for X: (6480, 31, 251, 1)
New shape for y: (6480,)
Train index for this split: [   1    3    5 ... 6476 6477 6479]
Number of samples for train set: 5184
Test index for this split: [   0    2    4 ... 6472 6474 6478]
Number of samples for test set: 1296
Epoch 1/250


ValueError: in user code:

    File "/home/malfonzo/akulejo/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/malfonzo/akulejo/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/malfonzo/akulejo/lib/python3.10/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/malfonzo/akulejo/lib/python3.10/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/home/malfonzo/akulejo/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/malfonzo/akulejo/lib/python3.10/site-packages/keras/engine/input_spec.py", line 295, in assert_input_compatibili

In [19]:
# PER SUB 
# SUBJECTS=[1,2,3,4,5,6,7,8,9,11,12,13,14,16,17]
# for temp_sub in SUBJECTS:
CLASSES= [2,3]
#questa va applicata a for loop di subject che deve essere il più esterno 
sub="{:02d}".format(temp_sub)
for n_classes in CLASSES:
    #Define loss function
    loss_fn= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    LABELS= list(range(0,n_classes))
    numero_classi=n_classes

    dir0= os.path.join("FINAL_motor_imagery_classification/10_sec_windows/raw/"+str(n_classes)+"class/output/", "all")
    os.mkdir(dir0)
    dir_input=("FINAL_motor_imagery_classification/10_sec_windows/raw/"+str(n_classes)+"class/input")

    evaluation=[]
    iteration=[]
    confusion_matrix_x_test=[]
    confusion_matrix_y_test= []
    validation_acc=[]
    PERFORMANCE=[]

#         print("SUBJECT: "+ str(sub))
    print("N_CLASSES: "+ str(n_classes))

#         X=sio.loadmat(dir_input+"/sub_"+str(sub)+ "_X.mat")["array_X"]
#         y= sio.loadmat(dir_input+"/sub_"+str(sub)+"_y.mat")["array_y"]
    X=sio.loadmat(dir_input+"/all_X.mat")["array_X"]
    y= sio.loadmat(dir_input+"/all_y.mat")["array_y"]
    X= X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
    y=y.flatten()
    print("New shape for X: " + str(X.shape))
    print("New shape for y: "+str(y.shape))

    dir1=os.path.join(dir0, "comparison")
    dir2=os.path.join(dir0, "temporal_convolution")
    dir3=os.path.join(dir0, "spatial_convolution")
    os.mkdir(dir1)
    os.mkdir(dir2)
    os.mkdir(dir3)

    ################################################################################################################################################################################################################################################################################

    n_folds = 5
    seed = 21
    shuffle_test = True
    EPOCHS=250

    kfold = KFold(n_splits = n_folds, shuffle = shuffle_test, random_state = seed)
    count=0
    sommatoria=0

    for train_index, test_index in kfold.split(X):
        count=count+1

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        print("Train index for this split: "+ str(train_index)) 
        print("Number of samples for train set: "+str(train_index.shape[0]))
        print("Test index for this split: "+ str(test_index))
        print("Number of samples for test set: "+str(test_index.shape[0]))

        # Define the model architecture - 

        model=Sequential()

        ##################################################################

        model.add(Conv2D(8, (1, 64), padding = 'same',
                                       input_shape = (N_chan, N_samples_long, 1),
                                       use_bias = False, name="temporal_conv"))
        model.add(BatchNormalization(name="batchnorm_1"))
        model.add(DepthwiseConv2D((31, 1), use_bias = False, 
                                       depth_multiplier = 2,
                                       depthwise_constraint = max_norm(1.), name="spatial_conv"))
        model.add(BatchNormalization(name="batchnorm_2"))
        model.add(Activation('elu', name="activation_1"))
        model.add(AveragePooling2D((1, 4), name="pooling_layer_1"))
        model.add(Dropout(0.5, name="dropout_1"))

        model.add(SeparableConv2D(16, (1, 16),
                                       use_bias = False, padding = 'same', name="separable_conv"))
        model.add(BatchNormalization(name="batchnorm_3"))
        model.add(Activation('elu', name="activation_2"))
        model.add(AveragePooling2D((1, 8), name="pooling_layer_2"))
        model.add(Dropout(0.5, name="drpout_2")) #QUI DROPOUT E' LASCIATO A 0.5 come in eegnet paper

        model.add(Flatten(name = 'flatten'))

        model.add(Dense(numero_classi, name = 'dense', 
                                 kernel_constraint = max_norm(0.25)))
        model.add(Activation('softmax', name = 'softmax'))


        # Define the optimizer
        optimizer= optimizers.Adam(
        learning_rate= 1e-3,
        weight_decay= 0
        )
        model.compile(optimizer=optimizer,
                       loss=loss_fn,
                       metrics=['accuracy'])

        evaluation.append(model.fit(X_train, y_train, batch_size=16,
                  epochs=EPOCHS, 
                  validation_data=(X_test, y_test), 
                  verbose=2, workers=1)
                     )

        # Iteration = fold, i am just saving the model for that fold
        iteration.append(model)

        confusion_matrix_x_test.append(X_test)
        confusion_matrix_y_test.append(y_test)

        #Plotting confusion matrix
        pred=model.predict(X_test)
        y_test_pred= np.argmax(pred, axis=1)

        confusion_matrix= metrics.confusion_matrix(y_test, y_test_pred, normalize='true')
        plt.figure()
        metrics.ConfusionMatrixDisplay(confusion_matrix).plot()
        plt.savefig(dir1+"/confusion_matrix_kfold_"+str(count))
        plt.close()

        validation_acc.append(np.sum(y_test==y_test_pred)/y_test.shape[0])

        PERFORMANCE.append(classification_report(y_test, y_test_pred, labels=LABELS, output_dict=True))

        #Salvo risultati di singolo fold
        sio.savemat(dir1+"/y_pred_test_kfold"+str(count), {"array": y_test_pred})
        sio.savemat(dir1+"/y_test_kfold"+str(count), {"array": y_test})


        # PLOTTO FILTRI TEMPORALI E SPAZIALI E LI SALVO
        var= (model.get_layer("temporal_conv").weights)
        for lallo in range(8):
            plt.figure()
            plt.title("temp_conv_"+str(lallo))
            plt.plot(var[0][0,:,0][:,lallo]) #this way i access the temporal filters, cambiando ultimo zero
            plt.savefig(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo))
            temp= (var[0][0,:,0][:,lallo]).numpy()
            sio.savemat(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": temp})
            plt.close()


        var_2= (model.get_layer("spatial_conv").weights)
        reshaped_var_2=tf.reshape(var_2[0][:,0,:,:],(31,16))
        for lallo in range(16):
            nump= reshaped_var_2[:,lallo].numpy()
            sio.savemat(dir3+"/spat_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": nump})

    ###################################################################################################################

    #plot accuracy and loss function across epochs
    epoch_vec=np.linspace(1,EPOCHS,EPOCHS)
    min_temp_loss=10
    min_temp_acc=10
    max_temp_loss=0
    max_temp_acc=0

    for idx in range(n_folds):
        if (np.min(evaluation[idx].history['loss'])<min_temp_loss):
            min_temp_loss=np.min(evaluation[idx].history['loss'])
        if (np.min(evaluation[idx].history['val_loss'])<min_temp_loss):
            min_temp_loss=np.min(evaluation[idx].history['val_loss'])
        if (np.max(evaluation[idx].history['loss'])>max_temp_loss):
            max_temp_loss=np.max(evaluation[idx].history['loss'])
        if (np.max(evaluation[idx].history['val_loss'])>max_temp_loss):
            max_temp_loss=np.max(evaluation[idx].history['val_loss'])

    for idx in range(n_folds):
        if (np.min(evaluation[idx].history['accuracy'])<min_temp_acc):
            min_temp_acc=np.min(evaluation[idx].history['accuracy'])
        if (np.min(evaluation[idx].history['val_accuracy'])<min_temp_acc):
            min_temp_acc=np.min(evaluation[idx].history['val_accuracy'])
        if (np.max(evaluation[idx].history['accuracy'])>max_temp_acc):
            max_temp_acc=np.max(evaluation[idx].history['accuracy'])
        if (np.max(evaluation[idx].history['val_accuracy'])>max_temp_acc):
            max_temp_acc=np.max(evaluation[idx].history['val_accuracy'])

    for idx in range(n_folds):
        loss_vec_train= evaluation[idx].history['loss']
        loss_vec_test= evaluation[idx].history['val_loss']

        plt.figure()
        plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
        plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Loss across epochs for fold: '+str(idx))
        plt.ylim([min_temp_loss, max_temp_loss])
        plt.legend()

        plt.savefig(dir1+"/loss_kfold_"+str(idx))
        plt.close()

    #plot accuracy and loss function across epochs
    epoch_vec=np.linspace(1,EPOCHS,EPOCHS)

    for idx in range(n_folds):
        loss_vec_train= evaluation[idx].history['accuracy']
        loss_vec_test= evaluation[idx].history['val_accuracy']

        plt.figure()
        plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
        plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.title('Accuracy across epochs for fold: '+str(idx))
        plt.ylim([min_temp_acc, max_temp_acc])
        plt.legend()

        plt.savefig(dir1+"/accuracy_kfold_"+str(idx))
        plt.close()

    #SALVO VARIABILI STATISTICHE E MODELLO IN MODO DA PLOTTARLO

#         acc_temp=[]
    accuratezza=np.zeros(n_classes*2)
#         for idx in range(n_folds):
#             acc_temp.append(PERFORMANCE[idx]["accuracy"])
    accuratezza[0]=(np.mean(validation_acc))
    accuratezza[1]=(statistics.pstdev(validation_acc))


    precisione=[]
    recall=[]
    f1_score=[]
    support=[]
    for classe in range(numero_classi):
        precision_temp=[]
        recall_temp=[]
        f1_score_temp=[]
        support_temp=[]
        for idx in range(n_folds):
            precision_temp.append(PERFORMANCE[idx][str(classe)]["precision"])
            recall_temp.append(PERFORMANCE[idx][str(classe)]["recall"])
            f1_score_temp.append(PERFORMANCE[idx][str(classe)]["f1-score"])
            support_temp.append(PERFORMANCE[idx][str(classe)]["support"])

        precisione.append(np.mean(precision_temp))
        precisione.append(statistics.pstdev(precision_temp))
        recall.append(np.mean(recall_temp))
        recall.append(statistics.pstdev(recall_temp))
        f1_score.append(np.mean(f1_score_temp))
        f1_score.append(statistics.pstdev(f1_score_temp))    
        support.append(np.mean(support_temp))
        support.append(statistics.pstdev(support_temp)) 

    sommario=[]
    sommario=np.vstack((accuratezza,precisione,recall,f1_score,support))
    sio.savemat(dir0+"/sommario.mat", {"array": sommario})

    gianfranco=model.predict(X)
    gianfranco2=np.argmax(gianfranco, axis=1)
    sio.savemat(dir0+"/predizione_totale.mat", {"array": gianfranco2})



N_CLASSES: 2
New shape for X: (4080, 31, 251, 1)
New shape for y: (4080,)
Train index for this split: [   0    1    3 ... 4077 4078 4079]
Number of samples for train set: 3264
Test index for this split: [   2   10   16   18   21   23   28   29   32   36   38   39   40   42
   43   58   65   72   75   76   86   87   88   99  102  103  110  117
  124  128  133  136  137  142  153  155  157  159  160  169  190  193
  200  204  209  210  212  215  216  217  219  221  224  232  244  258
  259  267  269  270  278  282  286  292  297  303  304  309  311  316
  319  324  328  332  333  335  342  346  347  351  360  362  365  369
  375  383  392  394  395  398  401  403  405  406  423  430  452  453
  455  457  459  465  467  469  480  484  487  490  497  504  512  513
  521  522  525  527  533  537  542  546  553  554  558  564  568  571
  574  575  579  581  592  601  607  615  616  622  637  642  646  657
  659  662  664  665  676  679  685  693  698  701  705  713  715  719
  721  730  731 

2023-04-04 08:58:25.683467: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_356/dropout_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


204/204 - 3s - loss: 0.6939 - accuracy: 0.5074 - val_loss: 0.6912 - val_accuracy: 0.5196 - 3s/epoch - 13ms/step
Epoch 2/250
204/204 - 1s - loss: 0.6908 - accuracy: 0.5199 - val_loss: 0.6854 - val_accuracy: 0.5613 - 944ms/epoch - 5ms/step
Epoch 3/250
204/204 - 1s - loss: 0.6698 - accuracy: 0.5809 - val_loss: 0.6605 - val_accuracy: 0.6140 - 947ms/epoch - 5ms/step
Epoch 4/250
204/204 - 1s - loss: 0.6456 - accuracy: 0.6333 - val_loss: 0.6526 - val_accuracy: 0.6078 - 963ms/epoch - 5ms/step
Epoch 5/250
204/204 - 1s - loss: 0.6410 - accuracy: 0.6425 - val_loss: 0.6382 - val_accuracy: 0.6483 - 918ms/epoch - 4ms/step
Epoch 6/250
204/204 - 1s - loss: 0.6254 - accuracy: 0.6621 - val_loss: 0.6330 - val_accuracy: 0.6458 - 924ms/epoch - 5ms/step
Epoch 7/250
204/204 - 1s - loss: 0.6201 - accuracy: 0.6584 - val_loss: 0.6278 - val_accuracy: 0.6507 - 940ms/epoch - 5ms/step
Epoch 8/250
204/204 - 1s - loss: 0.6149 - accuracy: 0.6731 - val_loss: 0.6236 - val_accuracy: 0.6605 - 947ms/epoch - 5ms/step
Epoch 

Epoch 66/250
204/204 - 1s - loss: 0.5184 - accuracy: 0.7525 - val_loss: 0.5759 - val_accuracy: 0.7022 - 958ms/epoch - 5ms/step
Epoch 67/250
204/204 - 1s - loss: 0.5291 - accuracy: 0.7426 - val_loss: 0.5786 - val_accuracy: 0.6838 - 946ms/epoch - 5ms/step
Epoch 68/250
204/204 - 1s - loss: 0.5221 - accuracy: 0.7537 - val_loss: 0.5766 - val_accuracy: 0.6985 - 945ms/epoch - 5ms/step
Epoch 69/250
204/204 - 1s - loss: 0.5177 - accuracy: 0.7506 - val_loss: 0.5808 - val_accuracy: 0.6936 - 960ms/epoch - 5ms/step
Epoch 70/250
204/204 - 1s - loss: 0.5236 - accuracy: 0.7494 - val_loss: 0.5655 - val_accuracy: 0.7059 - 958ms/epoch - 5ms/step
Epoch 71/250
204/204 - 1s - loss: 0.5163 - accuracy: 0.7503 - val_loss: 0.5677 - val_accuracy: 0.7108 - 979ms/epoch - 5ms/step
Epoch 72/250
204/204 - 1s - loss: 0.5282 - accuracy: 0.7457 - val_loss: 0.5767 - val_accuracy: 0.6949 - 977ms/epoch - 5ms/step
Epoch 73/250
204/204 - 1s - loss: 0.5245 - accuracy: 0.7515 - val_loss: 0.5753 - val_accuracy: 0.7096 - 953ms/e

Epoch 131/250
204/204 - 1s - loss: 0.5062 - accuracy: 0.7534 - val_loss: 0.5727 - val_accuracy: 0.7059 - 948ms/epoch - 5ms/step
Epoch 132/250
204/204 - 1s - loss: 0.4969 - accuracy: 0.7696 - val_loss: 0.5701 - val_accuracy: 0.7022 - 960ms/epoch - 5ms/step
Epoch 133/250
204/204 - 1s - loss: 0.5006 - accuracy: 0.7604 - val_loss: 0.5794 - val_accuracy: 0.6973 - 963ms/epoch - 5ms/step
Epoch 134/250
204/204 - 1s - loss: 0.4964 - accuracy: 0.7641 - val_loss: 0.5729 - val_accuracy: 0.7083 - 917ms/epoch - 4ms/step
Epoch 135/250
204/204 - 1s - loss: 0.5095 - accuracy: 0.7586 - val_loss: 0.5820 - val_accuracy: 0.7034 - 927ms/epoch - 5ms/step
Epoch 136/250
204/204 - 1s - loss: 0.5034 - accuracy: 0.7684 - val_loss: 0.5735 - val_accuracy: 0.6924 - 915ms/epoch - 4ms/step
Epoch 137/250
204/204 - 1s - loss: 0.5056 - accuracy: 0.7656 - val_loss: 0.5746 - val_accuracy: 0.7083 - 935ms/epoch - 5ms/step
Epoch 138/250
204/204 - 1s - loss: 0.4920 - accuracy: 0.7733 - val_loss: 0.5671 - val_accuracy: 0.7034 -

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

# POWERBANDS

In [8]:
# PER SUB 
# SUBJECTS=[1,2,3,4,5,6,7,8,9,11,12,13,14,16,17]
# for temp_sub in SUBJECTS:
# sub="{:02d}".format(temp_sub)

CLASSES= [2,3]
for n_classes in CLASSES:
    #Define loss function
    loss_fn= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    LABELS= list(range(0,n_classes))
    numero_classi=n_classes

#         dir0= os.path.join("FINAL_motor_imagery_classification/2_sec_windows/powerbands/"+str(n_classes)+"class/output/", "sub_"+str(sub))
    dir0= os.path.join("FINAL_motor_imagery_classification/2_sec_windows/powerbands/"+str(n_classes)+"class/output/", "all")

    os.mkdir(dir0)
    dir_input=("FINAL_motor_imagery_classification/2_sec_windows/powerbands/"+str(n_classes)+"class/input")

    evaluation=[]
    iteration=[]
    confusion_matrix_x_test=[]
    confusion_matrix_y_test= []
    validation_acc=[]
    PERFORMANCE=[]

    print("SUBJECT: "+ str(sub))
    print("N_CLASSES: "+ str(n_classes))

#     X=sio.loadmat(dir_input+"/sub_"+str(sub)+ "_X.mat")["array_X"]
#     y= sio.loadmat(dir_input+"/sub_"+str(sub)+"_y.mat")["array_y"]

    X=sio.loadmat(dir_input+"/all_X.mat")["array_X"]
    y= sio.loadmat(dir_input+"/all_y.mat")["array_y"]
    X= np.transpose(X, (0,1,3,2))
#         X=np.clip(X,0,100)
#         X= X.reshape(X.shape[0], X.shape[1], -1,1)

    y=y.flatten()
    print("New shape for X: " + str(X.shape))
    print("New shape for y: "+str(y.shape))

    dir1=os.path.join(dir0, "comparison")
    dir2=os.path.join(dir0, "temporal_convolution")
    dir3=os.path.join(dir0, "spatial_convolution")
    os.mkdir(dir1)
    os.mkdir(dir2)
    os.mkdir(dir3)

    ################################################################################################################################################################################################################################################################################

    n_folds = 5
    seed = 21
    shuffle_test = True
    EPOCHS=250

    N_chan=31
    N_samples_long=50
    N_frequency_bins= 4

    kfold = KFold(n_splits = n_folds, shuffle = shuffle_test, random_state = seed)
    count=0
    sommatoria=0

    for train_index, test_index in kfold.split(X):
        count=count+1

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        print("Train index for this split: "+ str(train_index)) 
        print("Number of samples for train set: "+str(train_index.shape[0]))
        print("Test index for this split: "+ str(test_index))
        print("Number of samples for test set: "+str(test_index.shape[0]))

        # Define the model architecture - 

#             model= EEGNet(n_classes, Chans = N_chan, Samples = N_samples_long, 
#              dropoutRate = Drop_test, kernLength = KernLength_test, F1 = F1_test, 
#              D = D_test, F2 = F2_test, norm_rate = 0.25, dropoutType = 'Dropout', )

        model=Sequential()

        ##################################################################

        model.add(Conv2D(8, (1, 64), padding = 'same',
                                       input_shape = (N_chan, N_samples_long, 4),
                                       use_bias = False, name="temporal_conv"))
        model.add(BatchNormalization(name="batchnorm_1"))
        model.add(DepthwiseConv2D((31, 1), use_bias = False, 
                                       depth_multiplier = 2,
                                       depthwise_constraint = max_norm(1.), name="spatial_conv"))
        model.add(BatchNormalization(name="batchnorm_2"))
        model.add(Activation('elu', name="activation_1"))
        model.add(AveragePooling2D((1, 4), name="pooling_layer_1"))
        model.add(Dropout(0.5, name="dropout_1"))

        model.add(SeparableConv2D(16, (1, 16),
                                       use_bias = False, padding = 'same', name="separable_conv"))
        model.add(BatchNormalization(name="batchnorm_3"))
        model.add(Activation('elu', name="activation_2"))
        model.add(AveragePooling2D((1, 8), name="pooling_layer_2"))
        model.add(Dropout(0.5, name="drpout_2")) #QUI DROPOUT E' LASCIATO A 0.5 come in eegnet paper

        model.add(Flatten(name = 'flatten'))

        model.add(Dense(numero_classi, name = 'dense', 
                                 kernel_constraint = max_norm(0.25)))
        model.add(Activation('softmax', name = 'softmax'))


        # Define the optimizer
        optimizer= optimizers.Adam(
        learning_rate= 1e-3,
        weight_decay= 0
        )
        model.compile(optimizer=optimizer,
                       loss=loss_fn,
                       metrics=['accuracy'])

        evaluation.append(model.fit(X_train, y_train, batch_size=16,
                  epochs=EPOCHS, 
                  validation_data=(X_test, y_test), 
                  verbose=2, workers=1)
                     )


        # Iteration = fold, i am just saving the model for that fold
        iteration.append(model)

        confusion_matrix_x_test.append(X_test)
        confusion_matrix_y_test.append(y_test)

        #Plotting confusion matrix
        pred=model.predict(X_test)
        y_test_pred= np.argmax(pred, axis=1)

        confusion_matrix= metrics.confusion_matrix(y_test, y_test_pred, normalize='true')
        plt.figure()
        metrics.ConfusionMatrixDisplay(confusion_matrix).plot()
        plt.savefig(dir1+"/confusion_matrix_kfold_"+str(count))
        plt.close()

        validation_acc.append(np.sum(y_test==y_test_pred)/y_test.shape[0])

        PERFORMANCE.append(classification_report(y_test, y_test_pred, labels=LABELS, output_dict=True))

        #Salvo risultati di singolo fold
        sio.savemat(dir1+"/y_pred_test_kfold"+str(count), {"array": y_test_pred})
        sio.savemat(dir1+"/y_test_kfold"+str(count), {"array": y_test})


        # PLOTTO FILTRI TEMPORALI E SPAZIALI E LI SALVO
        var= (model.get_layer("temporal_conv").weights)
        for lallo in range(8):
            plt.figure()
            plt.title("temp_conv_"+str(lallo))
            plt.plot(var[0][0,:,0][:,lallo]) #this way i access the temporal filters, cambiando ultimo zero
            plt.savefig(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo))
            temp= (var[0][0,:,0][:,lallo]).numpy()
            sio.savemat(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": temp})
            plt.close()


        var_2= (model.get_layer("spatial_conv").weights)
        reshaped_var_2=tf.reshape(var_2[0][:,0,:,:],(31,16))
        for lallo in range(16):
            nump= reshaped_var_2[:,lallo].numpy()
            sio.savemat(dir3+"/spat_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": nump})

    ###################################################################################################################

    #plot accuracy and loss function across epochs
    epoch_vec=np.linspace(1,EPOCHS,EPOCHS)
    min_temp_loss=10
    min_temp_acc=10
    max_temp_loss=0
    max_temp_acc=0

    for idx in range(n_folds):
        if (np.min(evaluation[idx].history['loss'])<min_temp_loss):
            min_temp_loss=np.min(evaluation[idx].history['loss'])
        if (np.min(evaluation[idx].history['val_loss'])<min_temp_loss):
            min_temp_loss=np.min(evaluation[idx].history['val_loss'])
        if (np.max(evaluation[idx].history['loss'])>max_temp_loss):
            max_temp_loss=np.max(evaluation[idx].history['loss'])
        if (np.max(evaluation[idx].history['val_loss'])>max_temp_loss):
            max_temp_loss=np.max(evaluation[idx].history['val_loss'])

    for idx in range(n_folds):
        if (np.min(evaluation[idx].history['accuracy'])<min_temp_acc):
            min_temp_acc=np.min(evaluation[idx].history['accuracy'])
        if (np.min(evaluation[idx].history['val_accuracy'])<min_temp_acc):
            min_temp_acc=np.min(evaluation[idx].history['val_accuracy'])
        if (np.max(evaluation[idx].history['accuracy'])>max_temp_acc):
            max_temp_acc=np.max(evaluation[idx].history['accuracy'])
        if (np.max(evaluation[idx].history['val_accuracy'])>max_temp_acc):
            max_temp_acc=np.max(evaluation[idx].history['val_accuracy'])

    for idx in range(n_folds):
        loss_vec_train= evaluation[idx].history['loss']
        loss_vec_test= evaluation[idx].history['val_loss']

        plt.figure()
        plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
        plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Loss across epochs for fold: '+str(idx))
        plt.ylim([min_temp_loss, max_temp_loss])
        plt.legend()

        plt.savefig(dir1+"/loss_kfold_"+str(idx))
        plt.close()

    #plot accuracy and loss function across epochs
    epoch_vec=np.linspace(1,EPOCHS,EPOCHS)

    for idx in range(n_folds):
        loss_vec_train= evaluation[idx].history['accuracy']
        loss_vec_test= evaluation[idx].history['val_accuracy']

        plt.figure()
        plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
        plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.title('Accuracy across epochs for fold: '+str(idx))
        plt.ylim([min_temp_acc, max_temp_acc])
        plt.legend()

        plt.savefig(dir1+"/accuracy_kfold_"+str(idx))
        plt.close()

    #SALVO VARIABILI STATISTICHE E MODELLO IN MODO DA PLOTTARLO

#         acc_temp=[]
    accuratezza=np.zeros(n_classes*2)
#         for idx in range(n_folds):
#             acc_temp.append(PERFORMANCE[idx]["accuracy"])
    accuratezza[0]=(np.mean(validation_acc))
    accuratezza[1]=(statistics.pstdev(validation_acc))


    precisione=[]
    recall=[]
    f1_score=[]
    support=[]
    for classe in range(numero_classi):
        precision_temp=[]
        recall_temp=[]
        f1_score_temp=[]
        support_temp=[]
        for idx in range(n_folds):
            precision_temp.append(PERFORMANCE[idx][str(classe)]["precision"])
            recall_temp.append(PERFORMANCE[idx][str(classe)]["recall"])
            f1_score_temp.append(PERFORMANCE[idx][str(classe)]["f1-score"])
            support_temp.append(PERFORMANCE[idx][str(classe)]["support"])

        precisione.append(np.mean(precision_temp))
        precisione.append(statistics.pstdev(precision_temp))
        recall.append(np.mean(recall_temp))
        recall.append(statistics.pstdev(recall_temp))
        f1_score.append(np.mean(f1_score_temp))
        f1_score.append(statistics.pstdev(f1_score_temp))    
        support.append(np.mean(support_temp))
        support.append(statistics.pstdev(support_temp)) 

    sommario=[]
    sommario=np.vstack((accuratezza,precisione,recall,f1_score,support))
    sio.savemat(dir0+"/sommario.mat", {"array": sommario})

    gianfranco=model.predict(X)
    gianfranco2=np.argmax(gianfranco, axis=1)
    sio.savemat(dir0+"/predizione_totale.mat", {"array": gianfranco2})



SUBJECT: 17
N_CLASSES: 2
New shape for X: (6480, 31, 50, 4)
New shape for y: (6480,)
Train index for this split: [   1    3    5 ... 6476 6477 6479]
Number of samples for train set: 5184
Test index for this split: [   0    2    4 ... 6472 6474 6478]
Number of samples for test set: 1296
Epoch 1/250


2023-04-05 23:34:13.247744: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_301/dropout_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


324/324 - 3s - loss: 0.6852 - accuracy: 0.5471 - val_loss: 0.6909 - val_accuracy: 0.5201 - 3s/epoch - 10ms/step
Epoch 2/250
324/324 - 1s - loss: 0.6609 - accuracy: 0.5982 - val_loss: 0.6887 - val_accuracy: 0.5332 - 1s/epoch - 4ms/step
Epoch 3/250
324/324 - 1s - loss: 0.6466 - accuracy: 0.6208 - val_loss: 0.7181 - val_accuracy: 0.4807 - 1s/epoch - 4ms/step
Epoch 4/250
324/324 - 1s - loss: 0.6235 - accuracy: 0.6426 - val_loss: 0.7148 - val_accuracy: 0.4815 - 1s/epoch - 4ms/step
Epoch 5/250
324/324 - 1s - loss: 0.6207 - accuracy: 0.6481 - val_loss: 0.7532 - val_accuracy: 0.4807 - 1s/epoch - 4ms/step
Epoch 6/250
324/324 - 1s - loss: 0.6061 - accuracy: 0.6674 - val_loss: 0.7605 - val_accuracy: 0.4807 - 1s/epoch - 4ms/step
Epoch 7/250
324/324 - 1s - loss: 0.6017 - accuracy: 0.6651 - val_loss: 0.7717 - val_accuracy: 0.4807 - 1s/epoch - 4ms/step
Epoch 8/250
324/324 - 1s - loss: 0.5978 - accuracy: 0.6775 - val_loss: 0.7418 - val_accuracy: 0.4807 - 1s/epoch - 5ms/step
Epoch 9/250
324/324 - 1s - 

Epoch 68/250
324/324 - 1s - loss: 0.5287 - accuracy: 0.7267 - val_loss: 0.7383 - val_accuracy: 0.4815 - 1s/epoch - 4ms/step
Epoch 69/250
324/324 - 1s - loss: 0.5273 - accuracy: 0.7236 - val_loss: 0.7606 - val_accuracy: 0.4838 - 1s/epoch - 4ms/step
Epoch 70/250
324/324 - 1s - loss: 0.5395 - accuracy: 0.7079 - val_loss: 0.6814 - val_accuracy: 0.5370 - 1s/epoch - 4ms/step
Epoch 71/250
324/324 - 1s - loss: 0.5187 - accuracy: 0.7321 - val_loss: 0.6772 - val_accuracy: 0.6890 - 1s/epoch - 4ms/step
Epoch 72/250
324/324 - 1s - loss: 0.5269 - accuracy: 0.7303 - val_loss: 0.6767 - val_accuracy: 0.6852 - 1s/epoch - 4ms/step
Epoch 73/250
324/324 - 1s - loss: 0.5278 - accuracy: 0.7257 - val_loss: 0.6881 - val_accuracy: 0.4969 - 1s/epoch - 4ms/step
Epoch 74/250
324/324 - 1s - loss: 0.5249 - accuracy: 0.7209 - val_loss: 0.7258 - val_accuracy: 0.4815 - 1s/epoch - 4ms/step
Epoch 75/250
324/324 - 1s - loss: 0.5307 - accuracy: 0.7294 - val_loss: 0.6850 - val_accuracy: 0.5031 - 1s/epoch - 4ms/step
Epoch 76

Epoch 134/250
324/324 - 1s - loss: 0.5134 - accuracy: 0.7274 - val_loss: 0.7769 - val_accuracy: 0.4823 - 1s/epoch - 4ms/step
Epoch 135/250
324/324 - 1s - loss: 0.5088 - accuracy: 0.7351 - val_loss: 1.0592 - val_accuracy: 0.4807 - 1s/epoch - 4ms/step
Epoch 136/250
324/324 - 1s - loss: 0.5190 - accuracy: 0.7315 - val_loss: 0.8056 - val_accuracy: 0.4815 - 1s/epoch - 4ms/step
Epoch 137/250
324/324 - 1s - loss: 0.5094 - accuracy: 0.7309 - val_loss: 1.1294 - val_accuracy: 0.4807 - 1s/epoch - 4ms/step
Epoch 138/250
324/324 - 1s - loss: 0.5138 - accuracy: 0.7224 - val_loss: 0.9006 - val_accuracy: 0.4815 - 1s/epoch - 4ms/step
Epoch 139/250
324/324 - 1s - loss: 0.5138 - accuracy: 0.7359 - val_loss: 1.1739 - val_accuracy: 0.4799 - 1s/epoch - 4ms/step
Epoch 140/250
324/324 - 1s - loss: 0.5125 - accuracy: 0.7218 - val_loss: 0.8405 - val_accuracy: 0.4823 - 1s/epoch - 4ms/step
Epoch 141/250
324/324 - 1s - loss: 0.5119 - accuracy: 0.7286 - val_loss: 0.8326 - val_accuracy: 0.4815 - 1s/epoch - 4ms/step


...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [9]:
# PER SUB 
# SUBJECTS=[1,2,3,4,5,6,7,8,9,11,12,13,14,16,17]
# for temp_sub in SUBJECTS:
# sub="{:02d}".format(temp_sub)

CLASSES= [2,3]
for n_classes in CLASSES:
    #Define loss function
    loss_fn= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    LABELS= list(range(0,n_classes))
    numero_classi=n_classes

#         dir0= os.path.join("FINAL_motor_imagery_classification/2_sec_windows/powerbands/"+str(n_classes)+"class/output/", "sub_"+str(sub))
    dir0= os.path.join("FINAL_motor_imagery_classification/10_sec_windows/powerbands/"+str(n_classes)+"class/output/", "all")

    os.mkdir(dir0)
    dir_input=("FINAL_motor_imagery_classification/10_sec_windows/powerbands/"+str(n_classes)+"class/input")

    evaluation=[]
    iteration=[]
    confusion_matrix_x_test=[]
    confusion_matrix_y_test= []
    validation_acc=[]
    PERFORMANCE=[]

    print("SUBJECT: "+ str(sub))
    print("N_CLASSES: "+ str(n_classes))

#     X=sio.loadmat(dir_input+"/sub_"+str(sub)+ "_X.mat")["array_X"]
#     y= sio.loadmat(dir_input+"/sub_"+str(sub)+"_y.mat")["array_y"]

    X=sio.loadmat(dir_input+"/all_X.mat")["array_X"]
    y= sio.loadmat(dir_input+"/all_y.mat")["array_y"]
    X= np.transpose(X, (0,1,3,2))
#         X=np.clip(X,0,100)
#         X= X.reshape(X.shape[0], X.shape[1], -1,1)

    y=y.flatten()
    print("New shape for X: " + str(X.shape))
    print("New shape for y: "+str(y.shape))

    dir1=os.path.join(dir0, "comparison")
    dir2=os.path.join(dir0, "temporal_convolution")
    dir3=os.path.join(dir0, "spatial_convolution")
    os.mkdir(dir1)
    os.mkdir(dir2)
    os.mkdir(dir3)

    ################################################################################################################################################################################################################################################################################

    n_folds = 5
    seed = 21
    shuffle_test = True
    EPOCHS=250

    N_chan=31
    N_samples_long=50
    N_frequency_bins= 4

    kfold = KFold(n_splits = n_folds, shuffle = shuffle_test, random_state = seed)
    count=0
    sommatoria=0

    for train_index, test_index in kfold.split(X):
        count=count+1

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        print("Train index for this split: "+ str(train_index)) 
        print("Number of samples for train set: "+str(train_index.shape[0]))
        print("Test index for this split: "+ str(test_index))
        print("Number of samples for test set: "+str(test_index.shape[0]))

        # Define the model architecture - 

#             model= EEGNet(n_classes, Chans = N_chan, Samples = N_samples_long, 
#              dropoutRate = Drop_test, kernLength = KernLength_test, F1 = F1_test, 
#              D = D_test, F2 = F2_test, norm_rate = 0.25, dropoutType = 'Dropout', )

        model=Sequential()

        ##################################################################

        model.add(Conv2D(8, (1, 64), padding = 'same',
                                       input_shape = (N_chan, N_samples_long, 4),
                                       use_bias = False, name="temporal_conv"))
        model.add(BatchNormalization(name="batchnorm_1"))
        model.add(DepthwiseConv2D((31, 1), use_bias = False, 
                                       depth_multiplier = 2,
                                       depthwise_constraint = max_norm(1.), name="spatial_conv"))
        model.add(BatchNormalization(name="batchnorm_2"))
        model.add(Activation('elu', name="activation_1"))
        model.add(AveragePooling2D((1, 4), name="pooling_layer_1"))
        model.add(Dropout(0.5, name="dropout_1"))

        model.add(SeparableConv2D(16, (1, 16),
                                       use_bias = False, padding = 'same', name="separable_conv"))
        model.add(BatchNormalization(name="batchnorm_3"))
        model.add(Activation('elu', name="activation_2"))
        model.add(AveragePooling2D((1, 8), name="pooling_layer_2"))
        model.add(Dropout(0.5, name="drpout_2")) #QUI DROPOUT E' LASCIATO A 0.5 come in eegnet paper

        model.add(Flatten(name = 'flatten'))

        model.add(Dense(numero_classi, name = 'dense', 
                                 kernel_constraint = max_norm(0.25)))
        model.add(Activation('softmax', name = 'softmax'))


        # Define the optimizer
        optimizer= optimizers.Adam(
        learning_rate= 1e-3,
        weight_decay= 0
        )
        model.compile(optimizer=optimizer,
                       loss=loss_fn,
                       metrics=['accuracy'])

        evaluation.append(model.fit(X_train, y_train, batch_size=16,
                  epochs=EPOCHS, 
                  validation_data=(X_test, y_test), 
                  verbose=2, workers=1)
                     )


        # Iteration = fold, i am just saving the model for that fold
        iteration.append(model)

        confusion_matrix_x_test.append(X_test)
        confusion_matrix_y_test.append(y_test)

        #Plotting confusion matrix
        pred=model.predict(X_test)
        y_test_pred= np.argmax(pred, axis=1)

        confusion_matrix= metrics.confusion_matrix(y_test, y_test_pred, normalize='true')
        plt.figure()
        metrics.ConfusionMatrixDisplay(confusion_matrix).plot()
        plt.savefig(dir1+"/confusion_matrix_kfold_"+str(count))
        plt.close()

        validation_acc.append(np.sum(y_test==y_test_pred)/y_test.shape[0])

        PERFORMANCE.append(classification_report(y_test, y_test_pred, labels=LABELS, output_dict=True))

        #Salvo risultati di singolo fold
        sio.savemat(dir1+"/y_pred_test_kfold"+str(count), {"array": y_test_pred})
        sio.savemat(dir1+"/y_test_kfold"+str(count), {"array": y_test})


        # PLOTTO FILTRI TEMPORALI E SPAZIALI E LI SALVO
        var= (model.get_layer("temporal_conv").weights)
        for lallo in range(8):
            plt.figure()
            plt.title("temp_conv_"+str(lallo))
            plt.plot(var[0][0,:,0][:,lallo]) #this way i access the temporal filters, cambiando ultimo zero
            plt.savefig(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo))
            temp= (var[0][0,:,0][:,lallo]).numpy()
            sio.savemat(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": temp})
            plt.close()


        var_2= (model.get_layer("spatial_conv").weights)
        reshaped_var_2=tf.reshape(var_2[0][:,0,:,:],(31,16))
        for lallo in range(16):
            nump= reshaped_var_2[:,lallo].numpy()
            sio.savemat(dir3+"/spat_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": nump})

    ###################################################################################################################

    #plot accuracy and loss function across epochs
    epoch_vec=np.linspace(1,EPOCHS,EPOCHS)
    min_temp_loss=10
    min_temp_acc=10
    max_temp_loss=0
    max_temp_acc=0

    for idx in range(n_folds):
        if (np.min(evaluation[idx].history['loss'])<min_temp_loss):
            min_temp_loss=np.min(evaluation[idx].history['loss'])
        if (np.min(evaluation[idx].history['val_loss'])<min_temp_loss):
            min_temp_loss=np.min(evaluation[idx].history['val_loss'])
        if (np.max(evaluation[idx].history['loss'])>max_temp_loss):
            max_temp_loss=np.max(evaluation[idx].history['loss'])
        if (np.max(evaluation[idx].history['val_loss'])>max_temp_loss):
            max_temp_loss=np.max(evaluation[idx].history['val_loss'])

    for idx in range(n_folds):
        if (np.min(evaluation[idx].history['accuracy'])<min_temp_acc):
            min_temp_acc=np.min(evaluation[idx].history['accuracy'])
        if (np.min(evaluation[idx].history['val_accuracy'])<min_temp_acc):
            min_temp_acc=np.min(evaluation[idx].history['val_accuracy'])
        if (np.max(evaluation[idx].history['accuracy'])>max_temp_acc):
            max_temp_acc=np.max(evaluation[idx].history['accuracy'])
        if (np.max(evaluation[idx].history['val_accuracy'])>max_temp_acc):
            max_temp_acc=np.max(evaluation[idx].history['val_accuracy'])

    for idx in range(n_folds):
        loss_vec_train= evaluation[idx].history['loss']
        loss_vec_test= evaluation[idx].history['val_loss']

        plt.figure()
        plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
        plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Loss across epochs for fold: '+str(idx))
        plt.ylim([min_temp_loss, max_temp_loss])
        plt.legend()

        plt.savefig(dir1+"/loss_kfold_"+str(idx))
        plt.close()

    #plot accuracy and loss function across epochs
    epoch_vec=np.linspace(1,EPOCHS,EPOCHS)

    for idx in range(n_folds):
        loss_vec_train= evaluation[idx].history['accuracy']
        loss_vec_test= evaluation[idx].history['val_accuracy']

        plt.figure()
        plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
        plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.title('Accuracy across epochs for fold: '+str(idx))
        plt.ylim([min_temp_acc, max_temp_acc])
        plt.legend()

        plt.savefig(dir1+"/accuracy_kfold_"+str(idx))
        plt.close()

    #SALVO VARIABILI STATISTICHE E MODELLO IN MODO DA PLOTTARLO

#         acc_temp=[]
    accuratezza=np.zeros(n_classes*2)
#         for idx in range(n_folds):
#             acc_temp.append(PERFORMANCE[idx]["accuracy"])
    accuratezza[0]=(np.mean(validation_acc))
    accuratezza[1]=(statistics.pstdev(validation_acc))


    precisione=[]
    recall=[]
    f1_score=[]
    support=[]
    for classe in range(numero_classi):
        precision_temp=[]
        recall_temp=[]
        f1_score_temp=[]
        support_temp=[]
        for idx in range(n_folds):
            precision_temp.append(PERFORMANCE[idx][str(classe)]["precision"])
            recall_temp.append(PERFORMANCE[idx][str(classe)]["recall"])
            f1_score_temp.append(PERFORMANCE[idx][str(classe)]["f1-score"])
            support_temp.append(PERFORMANCE[idx][str(classe)]["support"])

        precisione.append(np.mean(precision_temp))
        precisione.append(statistics.pstdev(precision_temp))
        recall.append(np.mean(recall_temp))
        recall.append(statistics.pstdev(recall_temp))
        f1_score.append(np.mean(f1_score_temp))
        f1_score.append(statistics.pstdev(f1_score_temp))    
        support.append(np.mean(support_temp))
        support.append(statistics.pstdev(support_temp)) 

    sommario=[]
    sommario=np.vstack((accuratezza,precisione,recall,f1_score,support))
    sio.savemat(dir0+"/sommario.mat", {"array": sommario})

    gianfranco=model.predict(X)
    gianfranco2=np.argmax(gianfranco, axis=1)
    sio.savemat(dir0+"/predizione_totale.mat", {"array": gianfranco2})



SUBJECT: 17
N_CLASSES: 2
New shape for X: (4080, 31, 50, 4)
New shape for y: (4080,)
Train index for this split: [   0    1    3 ... 4077 4078 4079]
Number of samples for train set: 3264
Test index for this split: [   2   10   16   18   21   23   28   29   32   36   38   39   40   42
   43   58   65   72   75   76   86   87   88   99  102  103  110  117
  124  128  133  136  137  142  153  155  157  159  160  169  190  193
  200  204  209  210  212  215  216  217  219  221  224  232  244  258
  259  267  269  270  278  282  286  292  297  303  304  309  311  316
  319  324  328  332  333  335  342  346  347  351  360  362  365  369
  375  383  392  394  395  398  401  403  405  406  423  430  452  453
  455  457  459  465  467  469  480  484  487  490  497  504  512  513
  521  522  525  527  533  537  542  546  553  554  558  564  568  571
  574  575  579  581  592  601  607  615  616  622  637  642  646  657
  659  662  664  665  676  679  685  693  698  701  705  713  715  719
  721

2023-04-06 00:26:14.707423: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_311/dropout_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


204/204 - 3s - loss: 0.6959 - accuracy: 0.4960 - val_loss: 0.6932 - val_accuracy: 0.4865 - 3s/epoch - 12ms/step
Epoch 2/250
204/204 - 1s - loss: 0.6920 - accuracy: 0.5123 - val_loss: 0.6934 - val_accuracy: 0.5061 - 868ms/epoch - 4ms/step
Epoch 3/250
204/204 - 1s - loss: 0.6921 - accuracy: 0.5101 - val_loss: 0.6929 - val_accuracy: 0.5331 - 849ms/epoch - 4ms/step
Epoch 4/250
204/204 - 1s - loss: 0.6921 - accuracy: 0.5224 - val_loss: 0.6928 - val_accuracy: 0.5049 - 889ms/epoch - 4ms/step
Epoch 5/250
204/204 - 1s - loss: 0.6866 - accuracy: 0.5401 - val_loss: 0.6924 - val_accuracy: 0.5123 - 905ms/epoch - 4ms/step
Epoch 6/250
204/204 - 1s - loss: 0.6874 - accuracy: 0.5251 - val_loss: 0.6933 - val_accuracy: 0.4926 - 871ms/epoch - 4ms/step
Epoch 7/250
204/204 - 1s - loss: 0.6853 - accuracy: 0.5328 - val_loss: 0.6931 - val_accuracy: 0.4975 - 905ms/epoch - 4ms/step
Epoch 8/250
204/204 - 1s - loss: 0.6817 - accuracy: 0.5576 - val_loss: 0.6917 - val_accuracy: 0.5049 - 909ms/epoch - 4ms/step
Epoch 

Epoch 66/250
204/204 - 1s - loss: 0.6275 - accuracy: 0.6360 - val_loss: 0.7133 - val_accuracy: 0.4963 - 851ms/epoch - 4ms/step
Epoch 67/250
204/204 - 1s - loss: 0.6357 - accuracy: 0.6278 - val_loss: 0.7112 - val_accuracy: 0.4939 - 872ms/epoch - 4ms/step
Epoch 68/250
204/204 - 1s - loss: 0.6267 - accuracy: 0.6409 - val_loss: 0.7302 - val_accuracy: 0.4975 - 895ms/epoch - 4ms/step
Epoch 69/250
204/204 - 1s - loss: 0.6288 - accuracy: 0.6290 - val_loss: 0.7186 - val_accuracy: 0.4963 - 885ms/epoch - 4ms/step
Epoch 70/250
204/204 - 1s - loss: 0.6244 - accuracy: 0.6376 - val_loss: 0.7197 - val_accuracy: 0.4963 - 898ms/epoch - 4ms/step
Epoch 71/250
204/204 - 1s - loss: 0.6337 - accuracy: 0.6198 - val_loss: 0.7244 - val_accuracy: 0.4951 - 889ms/epoch - 4ms/step
Epoch 72/250
204/204 - 1s - loss: 0.6357 - accuracy: 0.6311 - val_loss: 0.7029 - val_accuracy: 0.4939 - 863ms/epoch - 4ms/step
Epoch 73/250
204/204 - 1s - loss: 0.6327 - accuracy: 0.6354 - val_loss: 0.7184 - val_accuracy: 0.4963 - 867ms/e

Epoch 131/250
204/204 - 1s - loss: 0.6100 - accuracy: 0.6562 - val_loss: 0.7692 - val_accuracy: 0.4975 - 872ms/epoch - 4ms/step
Epoch 132/250
204/204 - 1s - loss: 0.6186 - accuracy: 0.6596 - val_loss: 0.7634 - val_accuracy: 0.4939 - 878ms/epoch - 4ms/step
Epoch 133/250
204/204 - 1s - loss: 0.6116 - accuracy: 0.6461 - val_loss: 0.7761 - val_accuracy: 0.4975 - 874ms/epoch - 4ms/step
Epoch 134/250
204/204 - 1s - loss: 0.6196 - accuracy: 0.6559 - val_loss: 0.7412 - val_accuracy: 0.4963 - 879ms/epoch - 4ms/step
Epoch 135/250
204/204 - 1s - loss: 0.6242 - accuracy: 0.6415 - val_loss: 0.7585 - val_accuracy: 0.4963 - 873ms/epoch - 4ms/step
Epoch 136/250
204/204 - 1s - loss: 0.6167 - accuracy: 0.6452 - val_loss: 0.7538 - val_accuracy: 0.4988 - 869ms/epoch - 4ms/step
Epoch 137/250
204/204 - 1s - loss: 0.6214 - accuracy: 0.6366 - val_loss: 0.7402 - val_accuracy: 0.4939 - 885ms/epoch - 4ms/step
Epoch 138/250
204/204 - 1s - loss: 0.6218 - accuracy: 0.6382 - val_loss: 0.7090 - val_accuracy: 0.4975 -

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>